In [ ]:
import pandas as pd
import gspread
import gspread_dataframe as gd
import os
from dotenv import load_dotenv
import psycopg2 as pc
import mysql.connector as mc

# Khai báo sheet đổ data

In [ ]:
mail = gspread.service_account('credentials.json')
open = mail.open_by_url('https://docs.google.com/spreadsheets/d/1GxDY1w_r_KZuWD568vFKwIYFwcxCHOw2tmf-aApqhjw/edit#gid=723640405')
sheet1 = open.worksheet('DATA RAW')

# Kết nối với database Odoo

In [ ]:
# Kết nối DB odoo mới
load_dotenv('.envnewodoo')
new_erp = pc.connect(
    host = os.getenv('new_host'),
    port = os.getenv('new_port'),
    database = os.getenv('new_db'),
    user = os.getenv('new_user'),
    password = os.getenv('new_password')
)

# Kết nối DB odoo cũ
load_dotenv('.envodoo')
old_erp = pc.connect(
    host = os.getenv('old_host'),
    port = os.getenv('old_port'),
    database = os.getenv('old_db'),
    user = os.getenv('old_user'),
    password = os.getenv('old_password')
)

In [ ]:
old_cursor = old_erp.cursor()
old_cursor.execute(
    """
SELECT
	DATE(sp.date_done + INTERVAL '7 hours') AS "date",
	pt.name AS pfcode,
	SPLIT_PART(pt.name,'-',1) AS default_code,
	SPLIT_PART(pt.name,'-',2) AS color,
	SPLIT_PART(pt.name,'-',3) AS size,
    pt.list_price as tag_price,
	SUM(sml.qty_done) - CASE WHEN qty_export IS NULL THEN 0 ELSE qty_export END AS qty_import
FROM stock_move_line sml
	LEFT JOIN stock_picking sp ON sml.picking_id = sp.id
	LEFT JOIN product_product pp ON sml.product_id = pp.id
	LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
	FULL JOIN
		(SELECT
			DATE(sp.date_done + INTERVAL '7 hours') AS "date",
			pp.id,
			SUM(sml.qty_done) AS qty_export
		FROM
			stock_move_line sml
			LEFT JOIN product_product pp ON sml.product_id = pp.id
			LEFT JOIN stock_picking sp ON sml.picking_id = sp.id
		WHERE
			sml.location_dest_id IN ('3823', '8') -- địa điểm đích là kho xử lý và vendor
			AND sml.location_id IN ('3835', '1') -- địa điểm nguồn là Kho tổng
			AND sml.state = 'done'
			AND sp.date_done + '7 hour' <= '2023-07-01'
		GROUP BY
			pp.id,
			DATE(sp.date_done + INTERVAL '7 hours')) AS export ON export.id = pp.id 
				AND export."date" = DATE(sp.date_done + INTERVAL '7 hours')
WHERE sml.location_dest_id IN ('3835', '1') -- địa điểm đích là kho tổng
	AND sml.location_id IN ('8', '3823') -- địa điểm nguồn là vendor, kho xử lý
	AND sml.state = 'done'
	AND sp.date_done + '7 hour' <= '2023-07-01'
GROUP BY
	DATE(sp.date_done + INTERVAL '7 hours'),
	pt.name,
    pt.list_price,
	qty_export""")

data_old = old_cursor.fetchall()
column_old = [name[0] for name in old_cursor.description]
df_old = pd.DataFrame(data_old,columns=column_old)
df_old

In [ ]:
cat_new_cursor = new_erp.cursor()
cat_new_cursor.execute(
    """
    SELECT
	DISTINCT(TRIM(SPLIT_PART(pc.complete_name,'/',1))) AS cat1,
	TRIM(SPLIT_PART(pc.complete_name,'/',2)) AS cat2,
	TRIM(SPLIT_PART(pc.complete_name,'/',3)) AS cat3,
	pc1.name AS collection,
	pp.default_code AS pfcode
FROM product_product pp
	LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
	LEFT JOIN product_category pc ON pt.categ_id=pc.id
	LEFT JOIN product_collection pc1 ON pt.collection_id=pc1.id
WHERE  pp.active = TRUE """)
data_cat_new = cat_new_cursor.fetchall()
columns_cat = [name[0] for name in cat_new_cursor.description]
df_cat_new = pd.DataFrame(data_cat_new,columns=columns_cat)
df_old = pd.merge(df_old, df_cat_new, on = 'pfcode', how = 'left')
df_old = df_old.reindex(['date','cat1','cat2','cat3','collection','pfcode','default_code', 'color', 'size','tag_price', 'qty_import'],axis=1)
df_old

In [ ]:
new_cursor = new_erp.cursor()
new_cursor.execute(
    """
SELECT 
	DATE(sp.date_done + INTERVAL '7 hours') AS "date",
	TRIM(SPLIT_PART(pc.complete_name,'/',1)) AS cat1,
	TRIM(SPLIT_PART(pc.complete_name,'/',2)) AS cat2,
	TRIM(SPLIT_PART(pc.complete_name,'/',3)) AS cat3,
    pc1.name as collection,
	pp.default_code AS pfcode,
	SPLIT_PART(pp.default_code,'-',1) AS default_code,
	SPLIT_PART(pp.default_code,'-',2) AS color,
	SPLIT_PART(pp.default_code,'-',3) AS size,
    pt.list_price as tag_price,
	SUM(sml.qty_done) - (CASE WHEN xuat.qty_export IS NULL THEN 0 ELSE xuat.qty_export END) AS qty_import
FROM stock_picking sp
	LEFT JOIN stock_move sm ON sp.id=sm.picking_id
	LEFT JOIN stock_picking_type spt ON sp.picking_type_id=spt.id
	LEFT JOIN stock_move_line sml ON sm.id=sml.move_id
	LEFT JOIN product_product pp ON sml.product_id=pp.id
	LEFT JOIN product_template pt ON pp.product_tmpl_id=pt.id
	LEFT JOIN product_category pc ON pt.categ_id=pc.id
	LEFT JOIN product_collection pc1 ON pt.collection_id=pc1.id
	FULL JOIN
		(SELECT 
			DATE(sp.date_done + INTERVAL '7 hours') AS "date",
            pp.default_code AS pfcode,
			SUM(sml.qty_done) AS qty_export
		FROM stock_picking sp
			LEFT JOIN stock_move sm ON sp.id=sm.picking_id
			LEFT JOIN stock_picking_type spt ON sp.picking_type_id=spt.id
			LEFT JOIN stock_move_line sml ON sm.id=sml.move_id
			LEFT JOIN product_product pp ON sml.product_id=pp.id
		WHERE sp.state = 'done'
			AND pp.active = TRUE
			AND sp.location_dest_id IN ('5597','8')
		GROUP BY pp.default_code, 
        	DATE(sp.date_done + INTERVAL '7 hours')) AS xuat ON xuat.pfcode = pp.default_code and xuat."date"=DATE(sp.date_done + INTERVAL '7 hours')
WHERE spt.name IN ('Nhận hàng')
	AND sp.location_dest_id = 5565
	AND sp.location_id = 8
	AND sp.state = 'done'
	AND pp.active = TRUE
GROUP BY	pp.default_code,
	xuat.qty_export,
	DATE(sp.date_done + INTERVAL '7 hours'),
	cat1,
	cat2,
	cat3,
    collection,
    pt.list_price
"""
)
data_new = new_cursor.fetchall()
column_new = [name[0] for name in new_cursor.description]
df_new = pd.DataFrame(data_new,columns=column_new)
df_new

In [ ]:
df_import = pd.concat([df_old,df_new],ignore_index=True)
df_import.sort_values(by=['date'],ascending=False,inplace=True)
print(df_import.shape)

In [ ]:
sheet1.batch_clear([f'A:K'])
gd.set_with_dataframe(sheet1,df_import,row=5,col=1)